# TensorFlow Hubを使ったテキスト分類

映画レビューのポジティブorネガティブの2クラス分類を行うサンプル。
このチュートリアルでは、TensorFlow HubとKerasを使った転移学習を行う。


50000件の映画レビューテキストが含まれるIMDB datasetを使用する。
25000件を訓練データとして、残りの25000件をテストデータとする。
なお、訓練データとテストデータのポジティブorネガティブレビューの件数はそれぞれ同じになっており、バランスが取れている。(つまり、不均衡データではない、ということ。)

このnotebookでは、TensorFlowでモデルを構築・訓練するのに`tf.keras`を使用する。そして、転移学習のライブラリ・プラットフォームであるTensorFlow Hubを用いる。

より詳細なテキスト分類のチュートリアルは下記のMLCC Text Classification Guideを参照すること。
https://developers.google.com/machine-learning/guides/text-classification/?hl=ja

In [1]:
import numpy as np

import tensorflow as tf

# !pip install -q tensorflow-hub
# !pip install -q tfds-nightly
# 代わりに以下のコマンドで事前に仮想環境にインストールしておく
# conda install tensorflow-hub
# conda install tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.8.0
GPU is available


## IMDB datasetのダウンロード

In [9]:
# # 15000件を訓練に、10000件をValidationに、25000件を評価に使用する
# チュートリアルのコードのままだとSlicing APIが動作しないようなので、
# splitの引数を別途定義して入力している
# 参考
# https://stackoverflow.com/questions/59959438/keyerror-invalid-split-train80-available-splits-are-train
split=[
    tfds.Split.TRAIN.subsplit(tfds.percent[:60]),
    tfds.Split.TRAIN.subsplit(tfds.percent[60:]),
    tfds.Split.TEST
]
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    # split=('train[:60%]', 'train[60%:]', 'test'),
    split=split,
    as_supervised=True)
print(train_data)
print(validation_data)
print(test_data)

<_OptionsDataset shapes: ((), ()), types: (tf.string, tf.int64)>
<_OptionsDataset shapes: ((), ()), types: (tf.string, tf.int64)>
<_OptionsDataset shapes: ((), ()), types: (tf.string, tf.int64)>


In [10]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by

In [11]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

# モデルの構築

テキストデータを表現するのにembedding Layerを使ってベクトルに変換する。

TensorFlow Hubには学習済みのtext embedding model(テキストtoベクトルのモデル)が用意されているので、それを利用する。  
今回使用するモデル
- https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1

その他の候補
- https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1
- https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1
- https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1

In [12]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                            dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [13]:
# レイヤーができたのでモデルに組み込む
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))

# 2層目のレイヤーのパラメータ数が336なのは
# 1層目のレイヤー出力20 x 2層目ユニット数16 + 2層目ユニット数分のバイアス16 = 336
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [15]:
# モデルのコンパイル
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=["accuracy"])

## モデルの訓練
512のサンプルを用いて20epoch分の訓練を行う。

In [26]:
history = model.fit(train_data.shuffle(10000).batch(512), 
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 138ms/step - loss: 0.7471 - accuracy: 0.5101 - val_loss: 0.6847 - val_accuracy: 0.5686
Epoch 2/20
30/30 [==============================] - 3s 98ms/step - loss: 0.6483 - accuracy: 0.6089 - val_loss: 0.6152 - val_accuracy: 0.6327
Epoch 3/20
30/30 [==============================] - 3s 95ms/step - loss: 0.5937 - accuracy: 0.6583 - val_loss: 0.5764 - val_accuracy: 0.6765
Epoch 4/20
30/30 [==============================] - 3s 100ms/step - loss: 0.5542 - accuracy: 0.6956 - val_loss: 0.5443 - val_accuracy: 0.7205
Epoch 5/20
30/30 [==============================] - 3s 95ms/step - loss: 0.5140 - accuracy: 0.7307 - val_loss: 0.5078 - val_accuracy: 0.7347
Epoch 6/20
30/30 [==============================] - 3s 98ms/step - loss: 0.4769 - accuracy: 0.7622 - val_loss: 0.4751 - val_accuracy: 0.7692
Epoch 7/20
30/30 [==============================] - 3s 95ms/step - loss: 0.4390 - accuracy: 0.7905 - val_loss: 0.4429 - val_accuracy: 0.7842
Epoch 8/20


## モデルの評価

In [28]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

loss: 0.321
accuracy: 0.854
